In [1]:
param_make = 'WORKHORSE'

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('./' + param_make + '/5.category.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_make = df['Make'].tolist()
list_year = df['Year'].tolist()
list_model = df['Model'].tolist()
list_engine = df['Engine'].tolist()
list_car_code = df['Car_Code'].tolist()
list_category = df['Category'].tolist()
list_category_code = df['Category_Code'].tolist()

print('总数量：' + str(len(list_category)))
list_category

总数量：1


['Engine']

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from urllib.parse import quote
import requests
import json

In [6]:
i = 0

make = list_make[0]
year = list_year[0]
model = list_model[0]
engine = list_engine[0]
car_code = list_car_code[0]
category = list_category[0]
category_code = list_category_code[0]

data = 'func=navnode_fetch&payload=' + quote('{"jsn":{"tab":"catalog","make":"') + make.replace(' ', '+') + quote('","year":"') + year + quote('","model":"') + model.replace('+', '%2B').replace(' ', '+').replace('&', '%26').replace('/', '%2F') + quote('","engine":"') + engine.replace(' ', '+').replace('/', '%2F') + quote('","carcode":"') + car_code + quote('","groupname":"') + category.replace(' ', '+').replace('&', '%26').replace('/', '%2F') + quote('","nodetype":"groupname"}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=data,
                     headers={'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'}).text

resp = json.loads(resp)

resp

{'html_fill_sections': {'navchildren[]': '<div class="ranavnode" id="nav[__GIP__1__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__1__]" value="{&quot;groupindex&quot;:&quot;__GIP__1__&quot;,&quot;tab&quot;:&quot;catalog&quot;,&quot;idepth&quot;:1,&quot;make&quot;:&quot;WORKHORSE&quot;,&quot;year&quot;:&quot;2009&quot;,&quot;model&quot;:&quot;W21&quot;,&quot;carcode&quot;:&quot;3297125&quot;,&quot;engine&quot;:&quot;8.1L V8&quot;,&quot;groupname&quot;:&quot;Engine&quot;,&quot;parttype&quot;:&quot;5340&quot;,&quot;nodetype&quot;:&quot;parttype&quot;}" /><input type="hidden" autocomplete="off" name="navnodeunique[catalog-workhorse-2009-w21-3297125-engine-5340]" id="navnodeunique[catalog-workhorse-2009-w21-3297125-engine-5340]" value="__GIP__1__" /><div id="nav_o[__GIP__1__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nvlineicon"></td><td class="niconspace ncollapsedicon" id="navicon[__GIP__1__]"><a href="/

In [7]:
from bs4 import BeautifulSoup

In [8]:
html = resp['html_fill_sections']['navchildren[]']

if html == '':
    print('fuck')

soup = BeautifulSoup(html, 'lxml')

with open('./type.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"catalog","idepth":1,"make":"WORKHORSE","year":"2009","model":"W21","carcode":"3297125","engine":"8.1L V8","groupname":"Engine","parttype":"5340","nodetype":"parttype"}'/>
   <input autocomplete="off" id="navnodeunique[catalog-workhorse-2009-w21-3297125-engine-5340]" name="navnodeunique[catalog-workhorse-2009-w21-3297125-engine-5340]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nvlineicon">
       </td>
       <td class="niconspace ncollapsedicon" id="navicon[__GIP__1__]">
        <a href="/en/catalog/workhorse,2009,w21,8.1l+v8,3297125,engine,oil+filter,5340" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <i

In [9]:
from lxml import etree

In [10]:
html = etree.HTML(str(soup))

list_type = html.xpath('//td[@class="nlabel"]/a/text()')

list_type

AttributeError: 'NoneType' object has no attribute 'xpath'

In [ ]:
list_url = html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')

list_type_code = [_.split(',')[-1] for _ in list_url]

list_type_code

In [ ]:
list_url = ['https://www.rockauto.com'+_ for _ in list_url]

list_url

In [ ]:
df_temp = pd.DataFrame({'status': 'ok',
                        'Make': make,
                        'Year': year,
                        'Model': model,
                        'Engine': engine,
                        'Car_Code': car_code,
                        'Category': category,
                        'Category_Code': category_code,
                        'Type': list_type,
                        'Type_Code': list_type_code,
                        'Url': list_url})

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df